<a href="https://colab.research.google.com/github/nandyc/langchain-rag-basics/blob/main/langchain_rag_QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nandyc/langchain-rag-basics.git

Cloning into 'langchain-rag-basics'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (9/9), 59.53 KiB | 539.00 KiB/s, done.
Resolving deltas: 100% (2/2), done.


In [2]:
%cd langchain-rag-basics

/content/langchain-rag-basics


In [3]:
%%capture
### Install required modules and set the envvar for Gemini API Key
!pip install pypdf2
!pip install chromadb
!pip install google.generativeai
!pip install langchain-google-genai
!pip install langchain
!pip install langchain_community

In [4]:
#Import Python modules
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.vectorstores import Chroma
from google.colab import userdata

In [5]:
#export GOOGLE_API_KEY="YOUR_GOOGLE_API_KEY"
GOOGLE_API_KEY =userdata.get('GEMINI_APIKEY')

In [6]:
#Load the models
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key = GOOGLE_API_KEY)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key = GOOGLE_API_KEY)

In [ ]:
#Load the PDF and create chunks
loader = PyPDFLoader("handbook.pdf")
text_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=250,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
pages = loader.load_and_split(text_splitter)

In [ ]:
#Turn the chunks into embeddings and store them in Chroma
vectordb=Chroma.from_documents(pages,embeddings)

In [ ]:
#Configure Chroma as a retriever with top_k=5
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

In [ ]:
#Create the retrieval chain
template = """
You are a helpful AI assistant.
Answer based on the context provided.
context: {context}
input: {input}
answer:
"""

In [ ]:
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

In [ ]:
#Invoke the retrieval chain
response=retrieval_chain.invoke({"input":"How do I apply for personal leave?"})

In [ ]:
#Print the answer to the question
print(response["answer"])